# GET URBAN TILE 
This notebook serves to read and elaborate urban data to retrieve 128x128 meter patch and save relevant .tif and .geojson for each tile, in preparation of the climate analysis 

In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box
from pathlib import Path
import numpy as np
import rasterio
from rasterio.transform import from_origin
from rasterio import features
import pandas as pd
import matplotlib.pyplot as plt
import json
import glob

Import data

In [3]:
#Load road network shapefile
road_path = Path("../dataset/BCN_GrafVial_Trams_ETRS89_SHP.shp")
roads_gdf = gpd.read_file(road_path)

# Project to EPSG:25831 (meters)
if roads_gdf.crs is None or roads_gdf.crs.to_epsg() != 25831:
    roads_gdf = roads_gdf.to_crs(epsg=25831)

# Keep only valid line geometries
#roads_gdf = roads_gdf[roads_gdf.geometry.type.isin(["LineString", "MultiLineString"])].copy()

# Load building footprints
buildings_path = Path("../dataset/Barcelona.geojson")
buildings_gdf = gpd.read_file(buildings_path)

# Project to EPSG:25831 to match roads
if buildings_gdf.crs is None or buildings_gdf.crs.to_epsg() != 25831:
    buildings_gdf = buildings_gdf.to_crs(epsg=25831)

# Load trees
tree_path = Path("../dataset//bcn_trees.geojson")
trees_gdf = gpd.read_file(tree_path)

# Ensure trees are projected to EPSG:25831
if trees_gdf.crs is None or trees_gdf.crs.to_epsg() != 25831:
    trees_gdf = trees_gdf.to_crs(epsg=25831)

# Clean height column
trees_gdf["height"] = pd.to_numeric(trees_gdf["height"], errors="coerce")
trees_gdf["height"] = trees_gdf["height"].fillna(0)

# Load LandUse
landuse_path = Path("../dataset/BCN_UsosSol_MOD.shp")
landuse_gdf = gpd.read_file(landuse_path)

# Project to EPSG:25831 (meters)
if landuse_gdf.crs is None or landuse_gdf.crs.to_epsg() != 25831:
    landuse_gdf = landuse_gdf.to_crs(epsg=25831)

buffer_path = Path("../dataset//25-0902 BCN Data/Buffered.shp")
buffer_gdf = gpd.read_file(buffer_path)

# Project to EPSG:25831 (meters)
if buffer_gdf.crs is None or buffer_gdf.crs.to_epsg() != 25831:
    buffer_gdf = buffer_gdf.to_crs(epsg=25831)

Sample road, by district, random or all. Optionally can visualize on the map

In [ ]:
#RANDOM ROADS IN SELECTED DISTRICT
#target_districts = {"01","02","10"}
#filtered_roads = roads_gdf[roads_gdf["Distric_D"].isin(target_districts)].copy()
#sampled_roads = filtered_roads.sample(n=200, random_state=42).copy()

#RANDOM ROADS
sampled_roads = roads_gdf.sample(n=20, random_state=42).copy()

# ALL ROADS
#sampled_roads = roads_gdf.copy()

# Create 128×128 m square patches around street midpoints
half_size = 64
windows = []
ids = []

for _, row in sampled_roads.iterrows():
    geom = row.geometry
    if geom.length == 0:
        continue
    midpoint = geom.interpolate(0.5, normalized=True)
    x, y = midpoint.x, midpoint.y
    square = box(x - half_size, y - half_size, x + half_size, y + half_size)
    windows.append(square)
    ids.append(row["C_Tram"])

windows_gdf = gpd.GeoDataFrame({'C_Tram': ids, 'geometry': windows}, crs=roads_gdf.crs)

# Save IDs to CSV
windows_gdf[["C_Tram"]].to_csv("../dataset/sampled_c_tram_ids.csv", index=False)

ax = windows_gdf.plot(facecolor="none", edgecolor="red", linewidth=1, figsize=(10, 10))
roads_gdf.plot(ax=ax, color="black", linewidth=0.5)
buildings_gdf.plot(ax=ax, facecolor="lightgray", edgecolor="gray", linewidth=0.2)
trees_gdf.plot(ax=ax, color="green", markersize=5, alpha=0.5)
buffer_gdf.plot(ax=ax, color="grey",alpha=0.5)
plt.title("Randomly Sampled 128×128 m Urban Patches")
plt.axis("off")
plt.tight_layout()
plt.show()

Plot the patches

In [ ]:
for i, window in enumerate(windows_gdf.geometry):
    fig, ax = plt.subplots(figsize=(6, 6))

    # Clip data to patch window
    clipped_roads = roads_gdf[roads_gdf.intersects(window)].copy().clip(window)
    clipped_buildings = buildings_gdf[buildings_gdf.intersects(window)].copy().clip(window)
    clipped_trees = trees_gdf[trees_gdf.intersects(window)].copy()
    clipped_buffer = buffer_gdf[buffer_gdf.intersects(window)].copy().clip(window)

    current_id = ids[i]
    relevant_buffer = buffer_gdf[buffer_gdf["C_Tram"] == current_id]
    clipped_buffer = gpd.clip(relevant_buffer[relevant_buffer.intersects(window)], window)
    
    # Compute building height
    if 'Z_MAX_VOL' in clipped_buildings.columns and 'Z_MIN_VOL' in clipped_buildings.columns:
        clipped_buildings["height"] = clipped_buildings["Z_MAX_VOL"] - clipped_buildings["Z_MIN_VOL"]
    else:
        clipped_buildings["height"] = 0

    # Plot patch boundary
    gpd.GeoSeries(window).boundary.plot(ax=ax, color="red", linestyle="--")

    # Plot buildings colored by height
    if not clipped_buildings.empty:
        clipped_buildings.plot(
            ax=ax,
            column="height",
            cmap="viridis",
            linewidth=0.5,
            edgecolor="gray",
            legend=True,
            legend_kwds={"label": "Building Height (m)", "shrink": 0.5},
            vmin=0,
            vmax=100
        )

    # Plot roads
    if not clipped_roads.empty:
        clipped_roads.plot(ax=ax, color="black", linewidth=1)

    # Plot tree canopies
    if not clipped_trees.empty:
        tree_canopies = clipped_trees.geometry.buffer(4.0)
        gpd.GeoSeries(tree_canopies).plot(
            ax=ax,
            facecolor="green",
            edgecolor="darkgreen",
            alpha=0.4,
            label="Tree Canopy"
        )

    if not clipped_buffer.empty:
        clipped_buffer.plot(ax=ax, color= "grey", alpha=0.5)

    ax.set_title(f"Patch {i+1} — Buildings + Roads + Trees", fontsize=11)
    ax.set_axis_off()
    plt.tight_layout()
    plt.show()

Create and save .tif and .geojson of the selected patches

In [ ]:
def export_geojson_with_top_keys(gdf, out_path, patch_id, features_type):
    geojson_dict = json.loads(gdf.to_json())
    geojson_dict["patch_id"] = patch_id
    geojson_dict["features_type"] = features_type
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(geojson_dict, f, indent=2)

# Output base path
output_dir = Path("../dataset/patches")
output_dir.mkdir(exist_ok=True)

# Grid resolution and size
grid_size = 128
patch_size = 128
pixel_size = patch_size / grid_size

for i, window in enumerate(windows_gdf.geometry):
    patch_id = windows_gdf.iloc[i]["C_Tram"]
    patch_name = f"patch_{i+1:04d}_{patch_id}"
    patch_path = output_dir / patch_name
    patch_path.mkdir(exist_ok=True)

    # Clip data
    clipped_roads = roads_gdf[roads_gdf.intersects(window)].copy().clip(window)
    clipped_roads["length_m"] = clipped_roads.geometry.length

    clipped_buildings = buildings_gdf[buildings_gdf.intersects(window)].copy().clip(window)
    clipped_trees = trees_gdf[trees_gdf.intersects(window)].copy()
    clipped_landuse = landuse_gdf[landuse_gdf.intersects(window)].copy().clip(window)

    relevant_buffer = buffer_gdf[buffer_gdf["C_Tram"] == patch_id]
    clipped_buffer = gpd.clip(relevant_buffer[relevant_buffer.intersects(window)], window)

    if "height" not in clipped_buildings.columns:
        if 'Z_MAX_VOL' in clipped_buildings.columns and 'Z_MIN_VOL' in clipped_buildings.columns:
            clipped_buildings["height"] = clipped_buildings["Z_MAX_VOL"] - clipped_buildings["Z_MIN_VOL"]
        else:
            clipped_buildings["height"] = 0

    minx, miny, maxx, maxy = window.bounds
    transform = from_origin(minx, maxy, pixel_size, pixel_size)

    dsm = np.zeros((grid_size, grid_size), dtype=np.float32)
    tree_dsm = np.zeros((grid_size, grid_size), dtype=np.float32)
    landuse_raster = np.zeros((grid_size, grid_size), dtype=np.uint8)

    for _, building in clipped_buildings.iterrows():
        if building.geometry.is_valid:
            height = building.get("height", 0)
            if height > 0:
                mask = features.geometry_mask(
                    [building.geometry], transform=transform,
                    invert=True, out_shape=dsm.shape
                )
                dsm[mask] = np.maximum(dsm[mask], height)

    for _, tree in clipped_trees.iterrows():
        if tree.geometry.is_valid:
            tree_height = tree.get("height", 5)
            if tree_height <= 0:
                continue
            canopy = tree.geometry.buffer(3.0)
            mask = features.geometry_mask(
                [canopy], transform=transform,
                invert=True, out_shape=tree_dsm.shape
            )
            tree_dsm[mask] = np.maximum(tree_dsm[mask], tree_height)

    def usos_to_code(val):
        if not val:
            return 0
        val = str(val).strip().lower()
        return {"urban_bloc": 1, "public_par": 2}.get(val, 0)

    if "Usos" in clipped_landuse.columns and clipped_landuse["Usos"].notna().any():
        shapes = [
            (row.geometry, usos_to_code(row["Usos"]))
            for _, row in clipped_landuse.iterrows()
            if row.geometry is not None and row.geometry.is_valid and usos_to_code(row["Usos"]) > 0
        ]
        if shapes:
            landuse_raster = features.rasterize(
                shapes=shapes, out_shape=landuse_raster.shape,
                transform=transform, fill=0, dtype="uint8",
                all_touched=False, merge_alg=features.MergeAlg.replace
            )

    buffer_mask = np.zeros((grid_size, grid_size), dtype=np.uint8)
    if not clipped_buffer.empty:
        mask = features.geometry_mask(
            clipped_buffer.geometry, transform=transform,
            invert=True, out_shape=buffer_mask.shape
        )
        buffer_mask[mask] = 1

    building_mask = np.zeros_like(landuse_raster, dtype=np.uint8)
    for _, building in clipped_buildings.iterrows():
        if building.geometry.is_valid:
            mask = features.geometry_mask(
                [building.geometry], transform=transform,
                invert=True, out_shape=building_mask.shape
            )
            building_mask[mask] = 1

    combined_landuse_raster = np.full_like(landuse_raster, 1, dtype=np.uint8)
    combined_landuse_raster[landuse_raster == 2] = 5
    combined_landuse_raster[building_mask == 1] = 2

    def write_tif(path, array, dtype, nodata=None):
        with rasterio.open(
            path, 'w', driver='GTiff', height=array.shape[0], width=array.shape[1],
            count=1, dtype=dtype, crs='EPSG:25831', transform=transform,
            compress='LZW', nodata=nodata
        ) as dst:
            dst.write(array, 1)

    # Patch stats
    tram_segment = roads_gdf[roads_gdf["C_Tram"] == patch_id]
    street_total_length = tram_segment.geometry.length.sum()
    buffer_area = clipped_buffer.geometry.area.sum()
    trees_in_buffer = clipped_trees[clipped_trees.centroid.within(clipped_buffer.union_all())]
    num_trees_in_buffer = len(trees_in_buffer)
    tree_canopies = trees_in_buffer.geometry.buffer(3.0)
    tree_canopy_area = tree_canopies.area.sum()
    green_area_ratio_in_buffer = tree_canopy_area / buffer_area if buffer_area > 0 else 0
    avg_tree_height_in_buffer = trees_in_buffer["height"].mean() if "height" in trees_in_buffer.columns else 10.0
    mean_building_height = clipped_buildings["height"].mean() if not clipped_buildings.empty else 0.0
    patch_area = window.area
    building_footprint_area = clipped_buildings.geometry.area.sum()
    building_coverage_ratio = building_footprint_area / patch_area if patch_area > 0 else 0

    if len(clipped_buildings) >= 2:
        distances = [
            clipped_buildings.drop(clipped_buildings.index[i]).geometry.distance(geom).min()
            for i, geom in enumerate(clipped_buildings.geometry)
        ]
        mean_inter_building_distance = np.mean(distances)
    else:
        mean_inter_building_distance = 0.0

    summary_data = {
        "C_Tram": patch_id,
        "street_total_length_m": round(street_total_length, 2),
        "buffer_area_m2": round(buffer_area, 2),
        "tree_canopy_area_in_buffer_m2": round(tree_canopy_area, 2),
        "num_trees_in_buffer": int(num_trees_in_buffer),
        "green_area_ratio_in_buffer": round(green_area_ratio_in_buffer, 2),
        "avg_tree_height_in_buffer": round(avg_tree_height_in_buffer, 2),
        "mean_building_height_in_patch": round(mean_building_height, 2),
        "building_coverage_ratio": round(building_coverage_ratio, 2),
        "mean_inter_building_distance_m": round(mean_inter_building_distance, 2)
    }

    pd.DataFrame([summary_data]).to_csv(patch_path / "patch_summary.csv", index=False)

    write_tif(patch_path / "dsm.tif", dsm, 'float32')
    write_tif(patch_path / "cdsm.tif", tree_dsm * (dsm == 0), 'float32')
    write_tif(patch_path / "dem.tif", np.zeros_like(dsm), 'float32')
    write_tif(patch_path / "landuse.tif", landuse_raster, 'uint8')
    write_tif(patch_path / "combined_landuse.tif", combined_landuse_raster, 'uint8')
    write_tif(patch_path / "buffer_mask.tif", buffer_mask, 'uint8', nodata=0)

    export_geojson_with_top_keys(clipped_landuse, patch_path / "landuse.geojson", patch_id, "LandUse")
    export_geojson_with_top_keys(clipped_buildings, patch_path / "buildings.geojson", patch_id, "Building")
    export_geojson_with_top_keys(clipped_roads, patch_path / "roads.geojson", patch_id, "Street")
    export_geojson_with_top_keys(clipped_trees, patch_path / "trees.geojson", patch_id, "Tree")
    export_geojson_with_top_keys(clipped_buffer, patch_path / "buffer.geojson", patch_id, "Buffer")
    export_geojson_with_top_keys(
        gpd.GeoDataFrame({'geometry': [window]}, crs=roads_gdf.crs),
        patch_path / "window.geojson", patch_id, "Window"
    )

    combined_patch_features = []
    for gdf_clip, label in [
        (clipped_buildings, "Building"),
        (clipped_roads, "Street"),
        (clipped_trees, "Tree"),
        (clipped_buffer, "Buffer")
    ]:
        if not gdf_clip.empty:
            gdf_clip = gdf_clip.copy()
            gdf_clip["features_type"] = label
            gdf_clip["patch_id"] = patch_id
            combined_patch_features.append(gdf_clip)

    combined_patch_features.append(
        gpd.GeoDataFrame(
            {'features_type': ["Window"], 'patch_id': [patch_id], 'geometry': [window]},
            crs=roads_gdf.crs
        )
    )

    gpd.GeoDataFrame(
        pd.concat(combined_patch_features, ignore_index=True),
        geometry="geometry", crs=roads_gdf.crs
    ).to_file(patch_path / "patch_combined.geojson", driver="GeoJSON")

    export_geojson_with_top_keys(
        buildings_gdf[buildings_gdf.intersects(window)].copy(),
        patch_path / "buildings_uncropped.geojson", patch_id, "Building"
    )

    fig, ax = plt.subplots(figsize=(6, 6))
    gpd.GeoSeries(window).boundary.plot(ax=ax, color="red", linestyle="--", linewidth=2)
    if not clipped_buildings.empty:
        clipped_buildings.plot(
            ax=ax,
            column="height",
            cmap="viridis",
            linewidth=0.5,
            edgecolor="gray",
            legend=True,
            legend_kwds={"label": "Building Height (m)", "shrink": 0.5},
            vmin=0,
            vmax=100
        )
    if not clipped_roads.empty:
        clipped_roads.plot(ax=ax, color="black", linewidth=1)
    if not clipped_trees.empty:
        gpd.GeoSeries(clipped_trees.geometry.buffer(3.0)).plot(
            ax=ax, facecolor="green", edgecolor="darkgreen", alpha=0.4
        )
    if not clipped_buffer.empty:
        clipped_buffer.plot(ax=ax, color="grey", alpha=0.5)

    ax.set_title(f"Patch {i+1} — patch_id: {patch_id}")
    ax.set_axis_off()
    plt.tight_layout()
    plt.savefig(patch_path / "patch.png", dpi=200, bbox_inches='tight')
    plt.close(fig)
    print(f"Saved patch {i+1:04d}")


Collect all the stats in a unique csv file

In [14]:
csv_paths = glob.glob(str(output_dir / "patch_*/patch_summary.csv"))
all_summaries = pd.concat([pd.read_csv(p) for p in csv_paths], ignore_index=True)
all_summaries.to_csv(output_dir / "all_patch_summaries.csv", index=False)